In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
import csv

%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
from os import listdir
from os.path import isfile, join
path = '../RetweetDataAOAS/retweet_data/'
root_tweet_names = [f for f in listdir(path) if isfile(join(path, f))]
num_tweets = len(root_tweet_names)

In [3]:
# Produces a dictionary of dataframes for each tweetfile, with initial 
# preprocessing
fields = ['RetweetCount', 'UserId', 'ScreenName', 'FollowerCount', 
          'DistanceFromRoot','Time', 'ParentScreenName', 'Text']
tweet_dfs = []
for i in range(num_tweets):
    tweet_df = pd.read_csv(path+root_tweet_names[i], sep="\t", header=None, 
                         quoting=csv.QUOTE_NONE, names=fields, encoding = "ISO-8859-1")
    
    tweet_df['Time'] = pd.to_datetime(tweet_df['Time'])

    screen_name_index = {}
    for index, row in tweet_df.iterrows():
        screen_name_index[row['ScreenName']] = index
    tweet_df['ParentDfIndex'] = tweet_df['ParentScreenName'].map(screen_name_index)
    
    tweet_dfs.append(tweet_df)

In [4]:
# Returns a dictionary of reaction times S_j^x keyed by user id
# Also modify the df to include a ReactionTime column
# S_j^x = T_j^x - T_P(j)^x is the time of the user's tweet minus its parent

def generate_reaction_times(tweet_df):
    reaction_times = {}
    for index, row in tweet_df.iterrows():
#         if index > 0: # ignore root tweet
        reaction_time = row['Time'] - tweet_df.at[row['ParentDfIndex'],
                                                  'Time']
        reaction_times[row['UserId']] = reaction_time
        tweet_df.loc[index,"ReactionTime"] = reaction_time
    return reaction_times

In [18]:
# modifies the dataframe to include M_j^x
# M_j^x for tweet x and user j is the number of j's followers who retweet x

def generate_number_of_follower_who_retweet(tweet_df):
    number_of_follower_who_retweet = {}
    
    for index, row in tweet_df.iterrows():
        parent_user_id = tweet_df.at[row['ParentDfIndex'], 'UserId']
        if parent_user_id not in number_of_follower_who_retweet:
            number_of_follower_who_retweet[parent_user_id] =1
        else:
            number_of_follower_who_retweet[parent_user_id] += 1
    
    # add to dataFrame
    for index, row in tweet_df.iterrows():
        if row['UserId'] in number_of_follower_who_retweet:
            count = number_of_follower_who_retweet[row['UserId']]
            tweet_df.loc[index,"FollowersRetweeted"] = count
        else: 
            tweet_df.loc[index,"FollowersRetweeted"] = 0

    return number_of_follower_who_retweet

In [21]:
log_s_j_x = []
for i in range(num_tweets):
    s_j_x = sorted(generate_reaction_times(tweet_dfs[i]).values())
    generate_number_of_follower_who_retweet(tweet_dfs[i])
    log_s_j_x.append([np.log(i.seconds) for i in s_j_x])

In [22]:
tweet_dfs[20]

,RetweetCount,UserId,ScreenName,FollowerCount,DistanceFromRoot,Time,ParentScreenName,Text,ParentDfIndex,ReactionTime,FollowersRetweeted
0,0,196917975,GarryShandling,186718,0,2012-04-13 22:56:53,GarryShandling,I'm a stay-at-home twitterer. That doesn't mak...,0,0 days 00:00:00,101.0
1,1,549558468,lpelly01,12,1,2012-04-13 23:51:40,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:54:47,0.0
2,2,26615944,cccarr,53,1,2012-04-13 23:53:05,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:56:12,0.0
3,3,21670816,MonaTalks,92,1,2012-04-13 23:53:13,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:56:20,0.0
4,4,32469012,markamadeo,10,1,2012-04-13 23:53:36,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:56:43,0.0
5,5,29562953,DALHOU,690,1,2012-04-13 23:53:37,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:56:44,0.0
6,6,434121852,Plewie,11,1,2012-04-13 23:54:11,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:57:18,0.0
7,7,539721796,NormanDeArmond,86,1,2012-04-13 23:54:11,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:57:18,0.0
8,8,433381152,TERRYATATE,86,1,2012-04-13 23:55:39,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:58:46,0.0
9,9,207932870,chriskkenny,3898,1,2012-04-13 23:55:51,GarryShandling,RT @GarryShandling: I'm a stay-at-home twitter...,0,0 days 00:58:58,0.0


In [29]:
# Scratch pymc3 code
with pm.Model() as twitter_model:
    # global model parameters
    # Time-related hyperparameters
    alpha = pm.Normal('alpha', mu=0, sd=100)
    sigma_squared_delta = pm.InverseGamma('sigma_squared_delta', alpha=0.5, beta=0.5)
    log_a_tau = pm.Normal('log_a_tau', mu=0, sd=10)
    b_tau = pm.Gamma('b_tau', alpha=1, beta=.002)
    
    # Graph-related hyperparameters
    sigma_squared_b = pm.InverseGamma('sigma_squared_b', alpha=0.5, beta=0.5)
    beta_0 = pm.Normal('beta_0', mu=0, tau=1/sigma_squared_b)
    beta_F = pm.Normal('beta_F', mu=0, tau=1/sigma_squared_b)
    beta_d = pm.Normal('beta_d', mu=0, tau=1/sigma_squared_b)
    
    # log-normal model for reaction times, nonrecursive...
    a_tau = np.exp(log_a_tau)
    tau_x_squared = pm.InverseGamma('tau_x_squared', alpha=a_tau, beta=b_tau, shape=num_tweets)
    alpha_x = pm.Normal('alpha_x', mu=alpha, tau=1/sigma_squared_delta, shape=num_tweets)
    log_rx_times = pm.Normal('log_s_j_x', mu=alpha_x, tau=tau_x_squared, observed=log_s_j_x)
    
    
    
    
    
    

ValueError: setting an array element with a sequence.